### Домашнее задание <a class="anchor" id="hw"></a><center>

In [2]:
import numpy as np
from sklearn.datasets import make_regression
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Реализуем класс узла

# Объявление класса Node. В дереве решений каждый узел
# представляет собой объект этого класса.
class Node:
    
    # Метод инициализации initit__, который вызывается при создании нового объекта класса Node.
    # Он принимает следующие параметры:
    # - index: индекс признака (колонки), по которому ведется сравнение с порогом.
    # - t: значение порога, с которым сравнивается значение признака.
    # - true_branch: поддерево, к которому переходят,
    # если условие (значение признака > порог) выполняется.
    # - false_branch: поддерево, к которому переходят, если условие не выполняется.
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index
        self.t = t
        self.true_branch = true_branch
        self.false_branch = false_branch

    # В итоге, класс Node определяет структуру узла дерева решений,
    # где каждый узел содержит информацию о том, по какому признаку
    # и с каким порогом проводится разделение,
    # а также ссылки на два поддерева — одно для случаев,
    # когда условие выполняется, и другое для случаев,
    # когда оно не выполняется.

In [4]:
# Расчет прироста

# Определение функции gain, которая принимает четыре параметра:
# - left_labels: метки классов в левом поддереве.
# - right_labels: метки классов в правом поддереве.
# - root_criterion: значение критерия на текущем (исходном) узле до разделения.
# - criterion: функция, вычисляющая значение критерия
# (например, Gini, энтропия) для набора меток классов.
def gain(left_labels, right_labels, root_criterion, criterion):

    # Вычисляет долю выборки, которая ушла в левое поддерево.
    # Для этого берется количество элементов в левом поддереве (left_labels.shape[0])
    # и делится на общее количество элементов (сумма элементов в левом и правом поддеревьях).
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    # Возвращает значение прироста (gain). Формула для вычисления прироста:
    # - root_criterion: значение критерия на текущем узле до разделения.
    # - p * criterion(left_labels): взвешенное значение критерия для левого поддерева.
    # - (1 - p) * criterion(right_labels): взвешенное значение критерия для правого поддерева.
    # Итоговый прирост получается вычитанием взвешенных значений критерия
    # для поддеревьев из значения критерия на текущем узле.
    return root_criterion - p * criterion(left_labels) - (1 - p) * criterion(right_labels)

In [5]:
# Разбиение датасета в узле

# Определение функции split, которая принимает четыре параметра:
# - data: матрица данных, где каждая строка — это объект, а каждый столбец — это признак.
# - labels: вектор меток классов для объектов.
# - column_index: индекс признака, по которому выполняется разбиение.
# - t: пороговое значение для разбиения.
def split(data, labels, column_index, t):
    
    # Создают два массива индексов:
    # - left: индексы объектов, у которых значение признака
    # в столбце column_index меньше или равно порогу t.
    # - right: индексы объектов, у которых значение признака
    # в столбце column_index больше порога t.
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)

    # Создают два новых массива данных:
    # - true_data: объекты, которые удовлетворяют условию
    # data[:, column_index] <= t (левая ветвь).
    # - false_data: объекты, которые не удовлетворяют условию (правая ветвь).
    true_data = data[left]
    false_data = data[right]
    
    # Создают два новых массива меток классов:
    # - true_labels: метки классов для объектов из true_data.
    # - false_labels: метки классов для объектов из false_data.
    true_labels = labels[left]
    false_labels = labels[right]
    
    # Возвращает четыре массива:
    # - true_data: объекты, удовлетворяющие условию.
    # - false_data: объекты, не удовлетворяющие условию.
    # - true_labels: метки классов для объектов из true_data.
    # - false_labels: метки классов для объектов из false_data.
    return true_data, false_data, true_labels, false_labels

# Таким образом, функция split разделяет датасет
# на два поддерева по заданному признаку и порогу,
# возвращая соответствующие данные
# и метки классов для каждого поддерева.

In [6]:
# Определение функции predict_object, которая принимает два параметра:
# - obj: объект (или пример) для которого нужно сделать предсказание.
# - node: текущий узел дерева решений,
# с которого начинается или продолжается предсказание.
def predict_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    # Проверяют, является ли текущий узел листом (Leaf):
    # - isinstance(node, Leaf): проверяет, является ли узел экземпляром класса Leaf.
    # - Если узел является листом, извлекается предсказание из узла (node.prediction),
    # и это предсказание возвращается как результат функции.
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    # Проверяет, удовлетворяет ли значение признака объекта в текущем узле условию:
    # - obj[node.index]: значение признака объекта в текущем узле (индекс признака node.index).
    # - <= node.t: проверка, меньше ли или равно ли это значение пороговому значению node.t.
    if obj[node.index] <= node.t:

        # Вызывается рекурсивная функция predict_object
        # для поддерева true_branch (левая ветвь).
        return predict_object(obj, node.true_branch)
    
    # Если значение признака объекта больше порога,
    # вызывается рекурсивная функция predict_object
    # для поддерева false_branch (правая ветвь).
    else:
        return predict_object(obj, node.false_branch)
    
# Таким образом, функция predict_object рекурсивно спускается по дереву решений,
# начиная с корневого узла и следуя по ветвям (левая или правая)
# в зависимости от значений признаков объекта, пока не достигнет листа.
# В листе функция возвращает предсказанное значение,
# соответствующее этому листу.

In [7]:
# Определение функции predict, которая принимает два параметра:
# - data: набор данных, для которого нужно сделать предсказания.
# Это может быть матрица, где каждая строка представляет собой объект (пример).
# - tree: дерево решений, используемое для предсказания.
def predict(data, tree):
    
    # Пустой список preds, в который будут добавляться
    # предсказания для каждого объекта из набора данных.
    preds = []

    for obj in data:

        # Для каждого объекта вызывается функция predict_object
        # с параметрами obj (текущий объект) и tree (дерево решений).
        # Функция predict_object возвращает предсказание для данного объекта,
        # которое сохраняется в переменной prediction.
        prediction = predict_object(obj, tree)

        # Добавляет предсказание prediction в список preds.
        preds.append(prediction)

    # После завершения цикла функция возвращает список preds,
    # который содержит предсказания для всех объектов из набора данных.
    return preds

# Таким образом, функция predict перебирает все объекты в наборе данных,
# используя дерево решений для предсказания класса каждого объекта,
# и собирает все предсказания в один список, который затем возвращает.

In [8]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index), '<=', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на отрицательном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

**1. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.**

In [9]:
# сгенерируем данные
data, targets = make_regression(n_features=2, n_informative=2, random_state=5)
# Этот код использует функцию make_regression из библиотеки sklearn.datasets
# для генерации синтетических данных для задачи регрессии. 
# Вызывает функцию make_regression и присваивает
# результаты двум переменным: data и targets.
# Параметры, переданные функции make_regression:
# - n_features=2: указывает, что у каждого объекта будет 2 признака.
# - n_informative=2: указывает, что оба признака будут информативными,
# то есть будут использоваться для вычисления целевой переменной (targets).
# - random_state=5: задает начальное значение для генератора случайных чисел,
# чтобы результаты были воспроизводимы.
# При использовании одного и того же random_state
# каждый запуск будет генерировать одинаковые данные.
# Теперь о возвращаемых значениях:
# - data: массив размером (n_samples, n_features) (по умолчанию n_samples=100),
# содержащий сгенерированные признаки. В данном случае, это матрица размером (100, 2).
# - targets: массив длиной n_samples, содержащий целевые значения (метки),
# соответствующие каждому объекту в data.

# Таким образом, функция make_regression создает синтетический набор данных
# для задачи регрессии с заданным числом признаков и информативных признаков,
# а также фиксирует случайное состояние для воспроизводимости результатов.

In [10]:
# Разобьем выборку на обучающую и тестовую

# Импортирует функцию train_test_split из модуля sklearn.model_selection.
# Функция используется для разделения массива данных на обучающую и тестовую выборки.
from sklearn.model_selection import train_test_split

# Вызывает функцию train_test_split и присваивает результат четырем переменным:
# - train_data_regr: данные для обучения (признаки).
# - test_data_regr: данные для тестирования (признаки).
# - train_target_regr: метки (целевые значения) для обучения.
# - test_target_regr: метки (целевые значения) для тестирования.
train_data_regr, test_data_regr, train_target_regr, test_target_regr = train_test_split(data, 
                                                                                        targets, 
                                                                                        test_size=0.3,
                                                                                        random_state=1)

# Параметры, переданные функции train_test_split:
# - data: исходный массив данных (признаки), который нужно разделить.
# - targets: исходный массив меток (целевые значения), который нужно разделить.
# - test_size=0.3: указывает, что 30% данных будет выделено
# на тестовую выборку, а оставшиеся 70% — на обучающую.
# - random_state=1: задает начальное значение для генератора случайных чисел,
# чтобы разделение данных было воспроизводимым.
# При использовании одного и того же random_state
# каждый запуск будет генерировать одинаковое разделение данных.

# Таким образом, функция train_test_split разбивает исходные данные и метки
# на обучающую и тестовую выборки в соответствии с заданным размером тестовой выборки
# и фиксированным состоянием генератора случайных чисел для воспроизводимости.

In [11]:
# И класс терминального узла (листа)
class Leaf:
    
    # Метод инициализаinitit__, который вызывается при создании
    # нового объекта класса Leaf. Он принимает два параметра:
    # - data: набор данных, попавших в данный лист.
    # - targets: соответствующие метки (целевые значения) для этих данных.
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

        # Вызывает метод predict() для объекта Leaf
        # и сохраняет результат в атрибут prediction.
        # Этот метод будет использоваться для предсказания
        # целевой переменной для данного листа.
        self.prediction = self.predict()
    
    # Метод predict, который определяет,
    # как вычислить предсказание для терминального узла (листа).
    def predict(self):

        # Возвращает среднее значение меток (целевых переменных) в узле Leaf
        # как предсказание для всех объектов, попавших в этот узел.
        return self.targets.mean()
    
# Таким образом, класс Leaf представляет собой терминальный узел дерева решений,
# который хранит данные и соответствующие метки для объектов,
# попавших в этот узел, и вычисляет предсказание для этих объектов.

In [12]:
# Определяет функцию для вычисления среднеквадратичной ошибки (MSE - Mean Squared Error).
def mse(targets):

    # Возвращает среднее значение квадрата разности
    # между каждым значением целевой переменной и их средним значением,
    # что является вычислением среднеквадратичной ошибки (MSE).
    return np.mean((targets - targets.mean())**2)

In [13]:
# Нахождение наилучшего разбиения

# Определение функции find_best_split, которая принимает два параметра:
# - data: массив признаков (наблюдений), который нужно разделить.
# - targets: массив целевых переменных (меток), соответствующих признакам.
def find_best_split(data, targets):
    
    #  обозначим минимальное количество объектов в узле
    min_samples_leaf = 5

    # Вычислить среднеквадратичную ошибку (MSE) для целевых переменных в корне дерева.
    root_mse = mse(targets)

    # Инициализируют переменные best_gain, best_t и best_index
    # для хранения лучшего прироста, порога и индекса признака для наилучшего разбиения.
    best_gain = 0
    best_t = None
    best_index = None
    
    # Количество признаков в наборе данных.
    n_features = data.shape[1]
    
    # Цикл перебирает индексы признаков.
    for index in range(n_features):

        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        # Вложенный цикл перебирает все уникальные значения признака.
        for t in t_values:

            # Вызывает функцию split, чтобы разделить данные и целевые переменные
            # на два поддерева на основе текущего признака и порога t.
            true_data, false_data, true_targets, false_targets = split(data, targets, index, t)

            #  Проверяет, достигается ли минимальное количество объектов в каждом поддереве.
            # Если нет, цикл продолжается без учета текущего разбиения.
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            # Вычисляет прирост качества (gain) для текущего разбиения, используя функцию gain.
            current_gain = gain(true_targets, false_targets, root_mse, mse)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    # Возвращает лучший прирост качества, соответствующий порог t
    # и индекс признака index для наилучшего разбиения.
    return best_gain, best_t, best_index

In [14]:
# Построение дерева с помощью рекурсивной функции

# Определение функции build_tree, которая принимает два параметра:
# - data: массив признаков (наблюдений), который нужно разделить.
# - target: массив целевых переменных (меток), соответствующих признакам.
def build_tree(data, target):

    # Вызывает функцию find_best_split для нахождения
    # наилучшего разбиения на текущем уровне дерева.
    # Она возвращает прирост в качестве разбиения (gain),
    # порог (t) и индекс признака (index),
    # на котором было найдено наилучшее разбиение.
    gain, t, index = find_best_split(data, target)

    # Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    # Проверяет, достигнут ли базовый случай,
    # когда нет прироста в качестве от разбиения.
    # Если прирост равен нулю, функция возвращает объект Leaf,
    # представляющий терминальный узел дерева.
    if gain == 0:
        return Leaf(data, target)

    # Вызывает функцию split, чтобы разделить данные и целевые переменные
    # на два поддерева на основе порога t и индекса признака index.
    true_data, false_data, true_target, false_target = split(data, target, index, t)

    # Рекурсивно вызывают функцию build_tree для построения двух поддеревьев:
    # одного для объектов, удовлетворяющих условию разбиения,
    # и другого для объектов, не удовлетворяющих этому условию.
    true_branch = build_tree(true_data, true_target)
    false_branch = build_tree(false_data, false_target)

    # Создает узел (node) с указанием индекса признака (index),
    # порога (t) и ссылок на поддеревья (true_branch и false_branch).
    node = Node(index, t, true_branch, false_branch)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return node

In [15]:
# Построим дерево по обучающей выборке
# Вызывает функцию build_tree, чтобы построить дерево решений
# на основе обучающих данных train_data_regr
# и соответствующих им меток train_target_regr.
# Результат (дерево) сохраняется в переменной my_tree.
my_tree = build_tree(train_data_regr, train_target_regr)
print_tree(my_tree)

Индекс 0 <= -0.10061434630710828
--> True:
  Индекс 0 <= -0.8568531547160899
  --> True:
    Прогноз: -109.75655471490919
  --> False:
    Индекс 0 <= -0.5732155560138283
    --> True:
      Прогноз: -54.35634172577482
    --> False:
      Индекс 1 <= -0.3058530211666308
      --> True:
        Прогноз: -29.105630694331246
      --> False:
        Прогноз: -10.772916465924025
--> False:
  Индекс 0 <= 0.9068894675659355
  --> True:
    Индекс 1 <= 0.6566194702604272
    --> True:
      Индекс 1 <= -1.0650326193820066
      --> True:
        Прогноз: 7.798014762375311
      --> False:
        Индекс 0 <= 0.41367880834311616
        --> True:
          Прогноз: 17.019366109004096
        --> False:
          Прогноз: 35.95087900163848
    --> False:
      Индекс 0 <= 0.34691932708774675
      --> True:
        Прогноз: 37.4238776327042
      --> False:
        Прогноз: 61.9558421220885
  --> False:
    Индекс 0 <= 1.3348485742415819
    --> True:
      Прогноз: 77.83232966482356
    --> F

In [16]:
# Импортирует функцию r2_score из модуля sklearn.metrics,
# которая используется для вычисления коэффициента детерминации R².
from sklearn.metrics import r2_score

# Вызывает функцию predict для получения предсказанных значений
# на обучающей выборке train_data_regr с использованием построенного дерева my_tree.
# Результат сохраняется в переменной train_answers.
train_answers = predict(train_data_regr, my_tree)

# Вычисляет коэффициент детерминации R² между
# истинными значениями целевой переменной на обучающей выборке train_target_regr
# и предсказанными значениями train_answers с использованием функции r2_score.
# Результат сохраняется в переменной train_r2.
train_r2 = r2_score(train_target_regr, train_answers)

print(train_r2)

# Вызывает функцию predict для получения предсказанных значений
# на тестовой выборке test_data_regr с использованием построенного дерева my_tree.
# Результат сохраняется в переменной answers.
answers = predict(test_data_regr, my_tree)

# Вычисляет коэффициент детерминации R² между истинными значениями целевой переменной
# на тестовой выборке test_target_regr и предсказанными значениями answers
# с использованием функции r2_score. Результат сохраняется в переменной test_r2.
test_r2 = r2_score(test_target_regr, answers)

print(test_r2)

0.9473030504970069
0.8558102546515577
